# DQN
Now that we know how to use neural networks in Tensorflow, lets implement our first RL algorithm to solve an RL environment. 

## LunarLander
The environment we will use is the OpenAI Lunar Lander environment. In this environment the agent must safely land a spaceship on the moon by carefully controlling the spaceship's thrusters. Below is a screenshot of the environment. Lets quickly evaluate a random agent on the environment to see how well it does.

![Lunar Lander](../images/lunarlander.png)

In [8]:
%%capture
# The line above hides the output of this cell.
# I do this for readability.

# Install dependencies
!pip install tensorflow
!pip install trfl
!pip install gym

In [2]:
import gym
import numpy as np

environment = gym.make("LunarLander-v2")
returns = []
for i in range(100):
    episode_return = 0
    observation = environment.reset()
    done = False
    while not done:
        action = environment.action_space.sample()

        next_observation, reward, done, info = environment.step(action)

        episode_return += reward

        observation = next_observation

    returns.append(episode_return)

print("Average Return for Random Agent:", np.average(returns))

Average Return for Random Agent: -201.8759844761783


So, a random agent on Lunar Lander gets a score of around -200. A winning score is +200. To create a winning agent we will implement DQN.

## DQN

In [10]:
import random
import copy

import tensorflow as tf
import trfl

class DQNAgent():

    def __init__(self, act_dim, hidden_size=256, learning_rate=1e-3, gamma=0.99,
        max_replay_size=10000, batch_size=256, epsilon_min=0.05, epsilon_dec=5e-6, 
        target_update=1000):
        self.num_actions = act_dim

        # A simple feed forward neural network
        # as our q-network
        self.q_network = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(hidden_size),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Dense(self.num_actions),
            ]
        )
        # The target q-network is just a copy of the online network
        self.target_q_network = copy.deepcopy(self.q_network)
        # The network optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        # Discount
        self.gamma = gamma
        # Super basic replay buffer
        self.batch_size = batch_size
        self.max_replay_size = max_replay_size
        self.replay_buffer = []
        self.replay_ctr = 0
        # Exploration
        self.epsilon = 1.0
        self.epsilon_min = epsilon_min
        self.epsilon_dec = epsilon_dec
        # Variables for periodically updating target
        self.learn_ctr = 0
        self.target_update = target_update

    def decrement_epsilon(self):
        # Decrement epsilon
        self.epsilon -= self.epsilon_dec # linear decay
        # You could experiment with exponential decay

        # If less than epsilon min, set to epsilon min
        if self.epsilon < self.epsilon_min:
            self.epsilon = self.epsilon_min

    def choose_action(self, observation):
        # Epsilon greedy: Choose random action.
        if np.random.random() < self.epsilon:
            action = np.random.choice(self.num_actions)
        
        # Epsilon greedy: Choose greedy action.
        else:
            # Convert numpy observation to tensor
            observation = tf.convert_to_tensor(observation)

            # Add batch dimension
            observation = tf.expand_dims(observation, axis=0)

            # Compute q-values
            q_values = self.q_network(observation)

            # Chose action with highest q-value
            action = tf.argmax(q_values, axis=1)

            # Convert tensor back to int
            action = action.numpy()[0]

        return action
        
    def update_target_network(self):
        # Periodicaly update the target network to be 
        # equal to the online network.
        # If update is too frequent, learning will be unstable.
        # If update is not frequent enough, learning will be slow.
        if self.learn_ctr % self.target_update == 0:
            self.target_q_network.set_weights(self.q_network.get_weights()) # hard update
            # You can experiment with a soft update (polyak averaging)


    def store(self, observation, action, reward, next_observation, done):
        # This method stores experience tuples in replay.
        experience_tuple = (observation, action, reward, next_observation, done)
        
        if self.replay_ctr < self.max_replay_size:
            self.replay_buffer.append(experience_tuple)
        else:
            # If replay is full, replace the oldest sample 
            # with new sample.
            idx = self.replay_ctr % self.max_replay_size
            self.replay_buffer[idx] = experience_tuple

        # Increment the replay counter.
        self.replay_ctr += 1

    def sample_replay(self):

        # Get a random batch from replay
        batch = random.sample(self.replay_buffer, self.batch_size) # super slow, make better

        # Convert list of tuples into tuple of lists.
        observations, actions, rewards, next_observations, dones = zip(*batch)

        # Convert to tensors
        observations = tf.convert_to_tensor(observations, dtype='float32')
        actions = tf.convert_to_tensor(actions, dtype='int32')
        rewards = tf.convert_to_tensor(rewards, dtype='float32')
        next_observations = tf.convert_to_tensor(next_observations, dtype='float32')
        dones = tf.convert_to_tensor(dones, dtype='float32')

        return observations, actions, rewards, next_observations, dones

    def learn(self):
        self.decrement_epsilon()

        # If too litle data in replay, do nothing.
        if len(self.replay_buffer) < self.batch_size:
            return

        # Sample from replay
        observations, actions, rewards, next_observations, dones = self.sample_replay()

        with tf.GradientTape() as tape:
            # Compute q-value of observation and action pair.
            q_values = self.q_network(observations)

            # We now need to select the q-value of the action the agent took.
            # `trfl` is a python package with useful, reusable RL methods. Check it out!! 
            # This method takes a tensor of q-values (shape=[B, num_actions]) and a 
            # tensor of actions (shape=[B]) and returns the q-values indexed by action
            # shape=[B], e.g. if q-values is [[2, 3, 5], [2, 6, 7]] and agent actions
            # are [2, 1] then the method returns [5, 6].
            q_value = trfl.indexing_ops.batched_index(q_values, actions)

            # Compute q-value of next observation using 
            # target network and `max` operator
            next_q_value = tf.reduce_max(self.target_q_network(next_observations), axis=1)

            # Bellman target
            target = rewards + (1 - dones) * self.gamma * next_q_value

            # Loss is just the mean square error
            # between the predicted q-value and the target.
            loss = tf.losses.MSE(target, q_value)

        # Get trainable variables
        variables = self.q_network.trainable_variables

        # Compute gradients with respect to trainable variables
        gradients = tape.gradient(loss, variables)

        # Apply gradients
        self.optimizer.apply_gradients(zip(gradients, variables))

        # Maybe update the target network weights
        self.update_target_network()

        # Increment the learn counter
        self.learn_ctr += 1

In [11]:
import datetime

# We define a class to log training progress to tensorboard.
class Logger():
    """
    An object to log data to tensorboard.
    """

    def __init__(self, logdir="./logs/"):
        current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        logdir = logdir + current_time

        self.summary_writer = tf.summary.create_file_writer(logdir)

    def write(self, step, logs):
        """Write logs to tensorboard.

        Args:
            step (Int): Training step of the logs.
            logs (Dict[str, float]): Dictionary of logs to be written to tensorboard.
        """
        with self.summary_writer.as_default():
            for key, value in logs.items():
                tf.summary.scalar(key, value, step=step)

In [12]:
def run_episode(environment, agent):
    """A function to run one episode.

    Args:
        environment (gym.environment): The gym environment to train the agent on.
        agent: The agent interacting with the environment.

    Returns:
        [float]: The return of the episode.
    """
    episode_return = 0
    done = False
    observation = environment.reset()
    while not done:
        # Choose action
        action = agent.choose_action(observation)

        # Step env
        next_observation, reward, done, info = environment.step(action)

        # Store experience
        agent.store(observation, action, reward, next_observation, done)

        # Update observation. Critical!!
        observation = next_observation

        # Add reward to episode return for logging.
        episode_return += reward

        # Agent learns at every step.
        # Dont need to wait for end of episode.
        agent.learn()
    
    return episode_return

In [19]:
def train(environment, agent, logger, num_iter=5000):
    """Funtion to train an agent in the environment.

    Args:
        environment: The environment.
        agent: The agent.
        logger: A tensorboard logger.
        num_iter (int, optional): Number of training episodes. Defaults to 5000.
    """
    scores = []
    for i in range(num_iter):
        score = run_episode(environment, agent)
        scores.append(score)

        # Stuff to write to tensorboard.
        logs = {
            "return": score,
            "epsilon": agent.epsilon # Lets log epsilon too!
        }
        # Write to tensorboard
        logger.write(step=i, logs=logs)

    return scores

In [26]:
%%capture
import gym

# Instantiate environment, agent and logger
environment = gym.make("LunarLander-v2")
act_dim = environment.action_space.n
agent = DQNAgent(act_dim)
logger = Logger()


In [28]:
# Run this cell if you want to monitor training with tensorboard.
# If tensorboard does not appear, run the cell a second time.
%load_ext tensorboard
%tensorboard --logdir logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 121172), started 0:00:02 ago. (Use '!kill 121172' to kill it.)

In [29]:
# Run this cell to start training.
returns = train(environment, agent, logger, num_iter=1000)

# Results

Here is a screen shot of the results after 1000 training episodes.
![DQN Results](../images/dqn_results.png)